In [61]:
import pandas as pd
import os
from openai import OpenAI
from dotenv import load_dotenv
from tqdm import tqdm
import time
import zlib
import base64
import json 

In [62]:
load_dotenv()

True

In [63]:
data = pd.read_excel(r"C:\Users\smrit\Downloads\NOS & OFQUAL\ofqual_sales_unitwise_with_markscheme.xlsx")


## Create Embeddings

In [64]:
#text-embedding-ada-002
#text-embedding-3-small


In [65]:
openai_client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  
)

In [66]:
def batch_list(data, batch_size):
    """Generate batches of data with a given batch size."""
    batches = []
    for i in range(0, len(data), batch_size):
        batch = data[i:i + batch_size]
        batches.append(batch)
    return batches

def get_batch_openai_embedding(texts: list, model="text-embedding-3-small", **kwargs):
    """
    Get embeddings of a batch of texts from OpenAI API.

    Args:
        texts (list): List of texts to get embeddings for.
        model (str): Model to use for embeddings.
        **kwargs: Additional arguments to pass to the OpenAI API.
    Returns:
        list[list]: List of embeddings of the texts.
    """    
    text_batches = batch_list(texts, 1024) if len(texts) > 1024 else [texts]
    embeddings = []
    for text_batch in text_batches:
        response = openai_client.embeddings.create(
            model=model,
            input=text_batch,
            **kwargs,
        )
        embeddings += [r.embedding for r in response.data]
    return embeddings

In [67]:
records = data.to_dict(orient='records')
record_batches = batch_list(records, 1000)

new_records = []
start_time = time.time()

for record_batch in tqdm(record_batches, desc="Processing batches", unit="batch"):
    text_values = [record['text'] for record in record_batch]
    embeddings = get_batch_openai_embedding(text_values)
    for embedding, record in zip(embeddings, record_batch):
        new_records.append({**record, 'embedding':embedding})

total_duration = time.time() - start_time
print(f"\nTotal processing time: {total_duration:.2f} seconds ({total_duration/60:.2f} minutes)")







Processing batches: 100%|█████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.46s/batch]


Total processing time: 2.47 seconds (0.04 minutes)


In [68]:
len(new_records)

56

## Create Vectors list

In [ ]:
#nos
vectors = [
    {
        'id': str(row['uuid']),  
        'values': row['embedding'],
        'metadata': {
            'nos_id': str(row['nos_id']) if pd.notna(row['nos_id']) else '',
            'industry': str(row['industry']) if pd.notna(row['industry']) else '',
            'title': str(row['title']) if pd.notna(row['title']) else '',
            'type': str(row['type']) if pd.notna(row['type']) else '',
            'text': str(row['text']) if pd.notna(row['text']) else ''
        }
    }
    for _, row in pd.DataFrame(new_records).iterrows()
]

# Add a check before upserting
print(f"First vector metadata sample: {vectors[0]['metadata']}")
print(f"Vector dimension: {len(vectors[0]['values'])}")

In [9]:
vectors = [
    {
        'id': str(row['uuid']),  
        'values': row['embedding'],         
        'metadata': {
            'level': str(int(float(row['level']))) if pd.notna(row['level']) else '',
            'text': str(row['text']) if pd.notna(row['text']) else '',
            'markscheme': str(row['markscheme']) if pd.notna(row['markscheme']) else ''
        }
    }
    for idx, (_, row) in enumerate(pd.DataFrame(new_records).iterrows())
]


In [10]:
print(f"First vector metadata sample: {vectors[0]['metadata']}")
print(f"Vector dimension: {len(vectors[0]['values'])}")

First vector metadata sample: {'level': '1', 'text': 'UNIT 1: Unit Title/Heading: Bus and Coach Electric/Hybrid Vehicle Awareness Unit Aim/Purpose: This unit is for people who work on or near Bus and Coach electric and hybrid vehicles but do not work on the vehicles high energy electrical system. The unit includes essential knowledge of the hazards associated with electric and hybrid vehicles and the precautions to follow to avoid these. Note: This unit does not deem someone competent to maintain, service or repair high energy electrical systems. Level: 1 Reference Number: R/618/3644 Credit Value: n/a Total Qualification Time (TQT): 13 Guided Learning Hours: 12 Learning Outcomes: 1. Understand the different types of Electric/Hybrid vehicles 2. Know and understand risks associated with Electric/Hybrid vehicles 3. Know and understand how to work safely with Electric/Hybrid vehicles Assessment Criteria: 1.1 Know how to identify Bus and Coach Electric/Hybrid vehicles 1.2 Know and understan

In [11]:
len(vectors)

110

### Compressing and decompressing the meta data


In [69]:
def compress_text(text):
    """Compress text using zlib and encode to base64 with JSON serialization."""
    if pd.isna(text):
        return ''
    # Convert to JSON string first
    json_str = json.dumps(text)
    compressed = zlib.compress(json_str.encode('utf-8'), level=9)
    return base64.b64encode(compressed).decode('utf-8')

# Keep data compressed in vectors
vectors = [
    {
        'id': str(row['uuid']),  
        'values': row['embedding'],
        'metadata': {
            'level': str(int(float(row['level']))) if pd.notna(row['level']) else '',
            'text': compress_text(row['text']) if pd.notna(row['text']) else '',
            'markscheme': compress_text(row['markscheme']) if pd.notna(row['markscheme']) else ''
        }
    }
    for idx, (_, row) in enumerate(pd.DataFrame(new_records).iterrows())
]

### querying ofqual data

In [59]:
def decompress_text(compressed_str):
    """Decompress text from base64 string back to original format."""
    if not compressed_str:
        return ''
    compressed_bytes = base64.b64decode(compressed_str.encode('utf-8'))
    decompressed = zlib.decompress(compressed_bytes)
    return json.loads(decompressed.decode('utf-8'))

# First define your query text
query_text = "key standards and regulations governing electrical installations"  

# Get the embedding for your query
query_vector = get_batch_openai_embedding([query_text])[0]

# Make the query to Pinecone
response = pinecone_index.search_items([query_vector], top_k=10)

# When processing query results
for match in response:
    print(f"Score: {match['score']}")
    print(f"Level: {match['metadata']['level']}")
    print(f"Text: {decompress_text(match['metadata']['text'])}")
    print(f"Markscheme: {decompress_text(match['metadata']['markscheme'])}\n")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:pinecone_index:Searching the index with 1 query vectors


Score: 0.587246835
Level: 3
Text: UNIT 1: Unit Title/Heading: Understand the requirements of electrical installations BS 7671:2018 Unit Aim/Purpose: This unit gives the learner an understanding of the full content of BS 7671, and how this applies to electrical installations within its scope. Level: Level 3 Reference Number: 602 Credit Value: Not explicitly stated Total Qualification Time (TQT): 40 Guided Learning Hours: 35 Learning Outcomes: 1. Understand the scope, object and fundamental principles of BS 7671. 2. Understand the definitions used within BS 7671. 3. Understand how to assess the general characteristics of electrical installations. 4. Understand requirements of protection for safety for electrical installations. 5. Understand the requirements for selection and erection of equipment for electrical installations. 6. Understand the requirements of inspection and testing of electrical installations. 7. Understand the requirements of special installations or locations as identi

In [13]:
os.chdir(r'C:\Users\smrit\Work\Kenpath\zavmo-api')

## Creating index and upsert vectors

In [60]:
#Index
#NOS: test-nos
#OFQAL: test-ofqal
#sales ofqual index :centrica-ofqual (energy)

In [35]:
os.getcwd()

'C:\\Users\\smrit\\Work\\Kenpath\\zavmo-api'

In [70]:
from pinecone_index import PineconeIndex

# Initialize PineconeIndex
pinecone_index = PineconeIndex(index_name='centrica-ofqual', dimension=1536)

INFO:pinecone_index:Connecting to existing index: centrica-ofqual


In [51]:
#pinecone_index.delete_all()

In [52]:
#delete index
#pinecone_index.delete_index()

In [71]:
# Upsert vectors into the index
pinecone_index.upsert_vectors(vectors, batch_size=100)

INFO:pinecone_index:Upserting 56 vectors into the index in batches of 100
Upserting batches: 100%|█████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.55s/it]


In [76]:
# Get the number of vectors in the index
vector_count = pinecone_index.get_vector_count()
print(f"Number of vectors in the index: {vector_count}")

INFO:pinecone_index:Total vectors in the index: 166


Number of vectors in the index: 166


### Querying 

In [ ]:
# Example query
query_vector = data['embeddings'][0]  
search_results = pinecone_index.search_items([query_vector])

# Print search results
for result in search_results:
    print(f"Found match: {result['id']} with score: {result['score']}, metadata: {result['metadata']}")

In [ ]:
#question = "I am a sales manager, what are the required skills I should know and learn according to NOS?"
competency = "Use sales-related information for planning and implementing sales activities"
query_vector = get_embedding(competency)

In [10]:
from pinecone import Pinecone
pinecone_client = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
index = pinecone_client.Index('test-ofqal')  
response = index.query(
    vector=query_vector,
    top_k=10,
    include_metadata=True,
    filter={"level":"level 3"}
)

In [47]:
from IPython.display import Markdown
Markdown("".join(list(response['matches'][0]['metadata']['text'])))


Organisational policy on data storage. Data protection. Legal and ethical issues of use of salesrelated information. GDPR compliant 2. Be able to obtain salesrelated information about customers, markets and competitors 2.1 Identify the information needed to develop knowledge about the organisation’s markets, customers and competitors Identify relevant information, on sector, competitors, own organisation policies, customers and product knowledge. 2.2 Identify sources of information about the organisation’s markets, customers and competitors Sector bodies, Industry bodies, Sector magazines, internet, Company’s house, networking events, colleagues, Competitor websites and sales brochures 2.3 Gather information about the organisation’s markets, customers and competitors Extract relevant information, list and update as required. Add to company database/CRM 3. Be able to use analytical tools and methods to provide salesrelated information 3.1 Select and use a variety of analytical tools and methods to analyse salesrelated information Know how to use appropriate software packages for analysing and presenting salesrelated information. CRM systems and reporting 3.2 Present your analysis of salesrelated information Identify trends in salesrelated data. Identify the target audience for

In [45]:
query_vector = get_embedding("Occupation relavant to Sales Executive")
search_results = pinecone_index.search_items([query_vector], top_k=100)

for result in search_results:
    print(f"Found match: {result['id']} with score: {result['score']}, metadata: {result['metadata']}")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:pinecone_index:Searching the index with 1 query vectors


Found match: 30517f01-4cc5-4271-ad68-edc25c56bc04 with score: 0.496667236, metadata: {'industry': 'Sales', 'nos_id': 'INSSAL022', 'text': 'Performance criteria\nYou must be able to:\n1.    identify the target markets for sales and prepare for sales activities \n2.    identify customers to contact and the range of products and services \n3.    prepare and follow call plans, email campaigns and other means of communication \nwith customers \n4.    help the customer become comfortable with making an investment into your \nproducts of services, understanding the value of doing so \n5.    prepare sales materials and accompanying messages during contacts with \ncustomers \n6.    adhere to your organisation’s dress code while selling to customers \n7.    agree procedures for collecting contact details of potential customers \n8.    adhere to health, safety and security requirements appropriate to the face-to-face \nsales environment \n9.    contact customers through relevant means of communic

In [3]:
os.chdir(r"C:\Users\Mumtaz Rahmani\OneDrive\Documents\projects\zavmo-api\zavmo\helpers")

### Retrieving nos documents passing currentrole and filter=Sales

In [7]:
import search
results = search.fetch_nos_text(industry="Sales", current_role="Sales Manager", top_k=100)

TypeError: fetch_nos_text() got an unexpected keyword argument 'top_k'

In [46]:
query_vector = get_embedding("Occupation relevant to Sales Executive")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [69]:
## Get NOS ID
index = pinecone_client.Index('test-nos')  
nos_searched_from_relavant_occupations = index.query(
        vector=query_vector,
        top_k=1,
        include_metadata=True,
        filter={"industry": "Sales", "type":"Developed by"},

    )
nos_id = nos_searched_from_relavant_occupations['matches'][0]['metadata']['nos_id']


## Get NOS sections
nos_sections_from_nos_id = index.query(
        vector=query_vector,
        top_k=2,
        include_metadata=True,
        filter={"nos_id": nos_id,  
                "$or": [
        {"type": "Performance criteria"},
        {"type": "Knowledge and understanding"}]
        })

matching_nos_doc = "\n".join([match['metadata']['text'] for match in nos_sections_from_nos_id['matches']])
matching_nos_doc



'Performance criteria\nYou must be able to:\n1.    identify the target markets for sales and prepare for sales activities \n2.    identify customers to contact and the range of products and services \n3.    prepare and follow call plans, email campaigns and other means of communication \nwith customers \n4.    help the customer become comfortable with making an investment into your \nproducts of services, understanding the value of doing so \n5.    prepare sales materials and accompanying messages during contacts with \ncustomers \n6.    adhere to your organisation’s dress code while selling to customers \n7.    agree procedures for collecting contact details of potential customers \n8.    adhere to health, safety and security requirements appropriate to the face-to-face \nsales environment \n9.    contact customers through relevant means of communication \n10.  identify customers’ requirements through questioning and confirm these by \nsummarising their needs and interests \n11.  emph

In [53]:
response['matches'][0]['metadata']

{'industry': 'Sales',
 'nos_id': 'INSSAL022',
 'text': 'Developed by\nInstructus\nVersion Number\n1\nDate Approved\n13 Feb 2024\nIndicative Review Date\n28 Feb 2029\nValidity\nCurrent\nStatus\nOriginal\nOriginating Organisation\nInstructus \nOriginal URN\nCFASAL014\nRelevant Occupations\nBusiness Sales Executives, Marketing and Sales Managers, \nSales Accounts and Business Development Managers, Sales \nRelated Occupations, Telephone Salespersons\nSuite\nSales\nKeywords\nSelling; sales opportunities; identify targets for sales; create \nprospects; sales procedures; assess delegate lists; verbal and \nnon-verbal communication; unique selling points; points of \ndifferentiation; collect customer testimonials; gain commitment \nfor sales; evaluate sales approach; time management; \npromotional materials; cross-selling and up-selling\nINSSAL022 \nSell products and services face-to-face\nINSSAL022 \nSell products and services face-to-face\n4\n',
 'title': 'Sell products and services face-to-

In [28]:
response['matches'][0]['metadata']


{'industry': 'Sales',
 'nos_id': 'INSSAL022',
 'text': 'Performance criteria\nYou must be able to:\n1.    identify the target markets for sales and prepare for sales activities \n2.    identify customers to contact and the range of products and services \n3.    prepare and follow call plans, email campaigns and other means of communication \nwith customers \n4.    help the customer become comfortable with making an investment into your \nproducts of services, understanding the value of doing so \n5.    prepare sales materials and accompanying messages during contacts with \ncustomers \n6.    adhere to your organisation’s dress code while selling to customers \n7.    agree procedures for collecting contact details of potential customers \n8.    adhere to health, safety and security requirements appropriate to the face-to-face \nsales environment \n9.    contact customers through relevant means of communication \n10.  identify customers’ requirements through questioning and confirm these

### Ofqual Querying

In [24]:
from zavmo.helpers.search import get_embedding

In [25]:
query_vector = get_embedding("what are qualification guidance for leadership management?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [26]:
#query with industry and level filters
from pinecone import Pinecone
pinecone_client = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
index = pinecone_client.Index('sales-ofqual')  
response = index.query(
    vector=query_vector,
    top_k=100,
    include_metadata=True,
    filter={
        "industry": "sales",
        "level": "7"  
    }
)


for match in response.matches:
    print(f"Score: {match.score}")
    print(f"Industry: {match.metadata['industry']}")
    print(f"Level: {match.metadata['level']}")
    print(f"Text: {match.metadata['text']}\n")

Score: 0.550926507
Industry: sales
Level: 7
Text: Unit aim: This unit aims to develop understanding and skills in the field of Leadership. The unit reviews the main approaches to leadership styles and looks at leadership in different contexts, including culture and gender. The relationship between leadership and strategy is emphasized. Level: 7 ISM Unit no: U753 RQF unit reference no: D/618/7423 Credit Value: 20 TQT: 200 Guided Learning Hours: 80 Learning outcomes: The learner will: 1. Understand the features and demands of leadership. Assessment criteria: The learner can: 1.1 Critically review traditional leadership styles. 1.2 Review the contingency approach to leadership. 1.3 Evaluate more recent approaches to leadership and their application in the workplace. Indicative content: 1.1 Introduction to leadership. The differences between management and leadership. “Great Man” theory. The trait theory of leadership. Trait theory and Emotional Intelligence. 1.2 The Behavioral view of lea

In [16]:
# Query without filter first to see what industries exist
response = index.query(
    vector=query_vector,
    top_k=10,
    include_metadata=True
)

# Print unique industries
industries = set()
for match in response.matches:
    industries.add(match.metadata.get('industry', ''))
print("Available industries:", industries)

Available industries: {'Fashion and Textiles', 'Fashion and Textiles England', 'Fashion and Textiles Level 3 Certificate', 'Fashion and Text iles England'}
